In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
# GET DAILY REVENUE FOR MOVIES THAT ARE IN CINEMAS

url = 'https://github.com/tjwaterman99/boxofficemojo-scraper/releases/latest/download/revenues_per_day.csv.gz'
revenue_df = pd.read_csv(url, parse_dates=['date'], index_col='id')

In [19]:
revenue_df.sort_values(by='date', ascending=False).head()

,date,title,revenue,theaters,distributor
id,,,,,
a6ab655a-be54-ae7e-a277-551c9b6160a5,2023-12-09,The Marvels,632723,1700.0,Walt Disney Studios Motion Pictures
c2e54537-c636-5422-cea7-73924b9c30a4,2023-12-09,Waitress: The Musical,1011265,1214.0,Bleecker Street Media
ef383ff1-f4ae-9e63-26fe-92c735ab7a66,2023-12-09,Die Hard2023 Re-release,354023,1355.0,Twenty First Century Entertainment
5a0f412a-5bb6-668b-bb13-77e2122db94d,2023-12-09,Saltburn,402649,707.0,Warner Bros.
2e53f83b-5088-3da0-9264-bd2b7f2d0bb8,2023-12-09,Thanksgiving,544427,2025.0,TriStar Pictures


In [4]:
r = pd.DataFrame()
r['date'] = pd.to_datetime(revenue_df['date'].values)

In [28]:
revenue_df[revenue_df['title']=='The Wonka']['date'].max().to_pydatetime()

NaT

In [29]:
pd.isna(revenue_df[revenue_df['title']=='The Wonka']['date'].max().to_pydatetime())

True

In [5]:
r['date'].min()- pd.DateOffset(months=1)

Timestamp('1999-12-01 00:00:00')

In [6]:
revenue_df[revenue_df['title'].str.startswith("The Hunger Games: The Ballad")].head()

,date,title,revenue,theaters,distributor
id,,,,,
30e8075e-9c92-fdc8-4248-bd612160ba10,2023-11-17,The Hunger Games: The Ballad of Songbirds & Sn...,19100000,3776.0,Lions Gate Films
762de0d5-943f-085b-d353-58b1ad368970,2023-11-18,The Hunger Games: The Ballad of Songbirds & Sn...,14651228,3776.0,Lions Gate Films
734919f9-4c1c-7d01-c42e-3164b5f58462,2023-11-19,The Hunger Games: The Ballad of Songbirds & Sn...,10907171,3776.0,Lions Gate Films
e288f437-adcb-a79c-2559-9a4dc5e0ac26,2023-11-20,The Hunger Games: The Ballad of Songbirds & Sn...,5266808,3776.0,Lions Gate Films
5fab426d-3367-d3fb-088b-d52c27c0c8b5,2023-11-21,The Hunger Games: The Ballad of Songbirds & Sn...,6500000,3776.0,Lions Gate Films


In [7]:
len(revenue_df)

345155

In [8]:
# CO POTRZEBA
# url = "https://api.themoviedb.org/3/trending/all/week?language=en-US&api_key={}".format("44f6c761b9c0466960bf14796a06db32")
# Wołać trending w loopie aż dostaniemy wszystkie strony
# zmergować w jedną tabelke
# Dla każdego trending movie dostać providerów i zmergować
# Stworzyć df z daily revenues
# joinować pierwszego dfa z box office df po tytule


In [9]:
def fetch_all_pages(api_key, endpoint, max_pages=5):
    url = f'https://api.themoviedb.org/3/{endpoint}'
    params = {'api_key': api_key, 'page': 1}
    response = requests.get(url, params=params)
    data = response.json()
    total_pages = 1
    if 'total_pages' in data:
        total_pages = min(data['total_pages'], max_pages)


    all_results = data['results']

    for page in range(2, total_pages + 1):
        params['page'] = page
        response = requests.get(url, params=params)
        data = response.json()
        all_results.extend(data['results'])

    return all_results

In [10]:
# Example usage
api_key = '44f6c761b9c0466960bf14796a06db32'
movies = fetch_all_pages(api_key, 'trending/all/week')
print(movies)

[{'adult': False, 'backdrop_path': '/1X7vow16X7CnCoexXh4H4F2yDJv.jpg', 'id': 466420, 'title': 'Killers of the Flower Moon', 'original_language': 'en', 'original_title': 'Killers of the Flower Moon', 'overview': 'When oil is discovered in 1920s Oklahoma under Osage Nation land, the Osage people are murdered one by one—until the FBI steps in to unravel the mystery.', 'poster_path': '/dB6Krk806zeqd0YNp2ngQ9zXteH.jpg', 'media_type': 'movie', 'genre_ids': [80, 18, 36], 'popularity': 2277.914, 'release_date': '2023-10-18', 'video': False, 'vote_average': 7.671, 'vote_count': 1314}, {'adult': False, 'backdrop_path': '/fm6KqXpk3M2HVveHwCrBSSBaO0V.jpg', 'id': 872585, 'title': 'Oppenheimer', 'original_language': 'en', 'original_title': 'Oppenheimer', 'overview': "The story of J. Robert Oppenheimer's role in the development of the atomic bomb during World War II.", 'poster_path': '/8Gxv8gSFCU0XGDykEGv7zR1n2ua.jpg', 'media_type': 'movie', 'genre_ids': [18, 36], 'popularity': 956.285, 'release_date

In [11]:
len(movies)

100

In [12]:
movies[1]

{'adult': False,
 'backdrop_path': '/fm6KqXpk3M2HVveHwCrBSSBaO0V.jpg',
 'id': 872585,
 'title': 'Oppenheimer',
 'original_language': 'en',
 'original_title': 'Oppenheimer',
 'overview': "The story of J. Robert Oppenheimer's role in the development of the atomic bomb during World War II.",
 'poster_path': '/8Gxv8gSFCU0XGDykEGv7zR1n2ua.jpg',
 'media_type': 'movie',
 'genre_ids': [18, 36],
 'popularity': 956.285,
 'release_date': '2023-07-19',
 'video': False,
 'vote_average': 8.139,
 'vote_count': 5345}

In [13]:
trending_df = pd.DataFrame(movies)

In [14]:
trending_df["common_name"] = trending_df.apply(lambda row: row["name"] if pd.notnull(row["name"]) else row["title"], axis=1)

In [30]:
trending_df.head()

,adult,backdrop_path,id,title,original_language,original_title,overview,poster_path,media_type,genre_ids,popularity,release_date,video,vote_average,vote_count,name,original_name,first_air_date,origin_country,common_name
0,False,/1X7vow16X7CnCoexXh4H4F2yDJv.jpg,466420,Killers of the Flower Moon,en,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,/dB6Krk806zeqd0YNp2ngQ9zXteH.jpg,movie,"[80, 18, 36]",2277.914,2023-10-18,False,7.671,1314,NaN,NaN,NaN,NaN,Killers of the Flower Moon
1,False,/fm6KqXpk3M2HVveHwCrBSSBaO0V.jpg,872585,Oppenheimer,en,Oppenheimer,The story of J. Robert Oppenheimer's role in t...,/8Gxv8gSFCU0XGDykEGv7zR1n2ua.jpg,movie,"[18, 36]",956.285,2023-07-19,False,8.139,5345,NaN,NaN,NaN,NaN,Oppenheimer
2,False,/vcFW09U4834DyFOeRZpsx9x1D3S.jpg,57243,NaN,en,NaN,The Doctor is a Time Lord: a 900 year old alie...,/4edFyasCrkH4MKs6H4mHqlrxA6b.jpg,tv,"[10759, 18, 10765]",647.093,NaN,NaN,7.461,2721,Doctor Who,Doctor Who,2005-03-26,[GB],Doctor Who
3,False,/1pmLsPXz5k37BILPH1wq1OV2E7U.jpg,239770,NaN,en,NaN,The Doctor and friends travel from the dawn of...,/2I8aMfUvgRKQvEpBIQVKMbXgMsi.jpg,tv,"[10759, 18, 10765]",268.284,NaN,NaN,7.900,42,Doctor Who,Doctor Who,,[GB],Doctor Who
4,False,/yOm993lsJyPmBodlYjgpPwBjXP9.jpg,787699,Wonka,en,Wonka,Willy Wonka – chock-full of ideas and determin...,/qhb1qOilapbapxWQn9jtRCMwXJF.jpg,movie,"[35, 10751, 14]",1174.577,2023-12-06,False,7.052,96,NaN,NaN,NaN,NaN,Wonka


In [31]:
print(trending_df['media_type'].unique())

['movie' 'tv']


In [32]:
# Example usage
providers = []
for roww in trending_df.iterrows():
    row = roww[1]
    id = row["id"]
    type = row["media_type"]
    print(id, type)
    try:
        movie_providers = fetch_all_pages(api_key, f'{type}/{id}/watch/providers')
        if "PL" in movie_providers:
            movie_providers["PL"]["id"] = id
            providers.append(movie_providers["PL"])
    except:
        pass

466420 movie
872585 movie
57243 tv
239770 tv
787699 movie
609681 movie
670292 movie
1429 tv
695721 movie
202411 tv
507089 movie
335977 movie
96648 tv
37854 tv
901362 movie
753342 movie
346698 movie
575264 movie
84958 tv
897087 movie
1075794 movie
1022796 movie
840430 movie
95479 tv
726209 movie
800158 movie
204082 tv
926393 movie
572802 movie
569094 movie
94244 tv
945729 movie
798021 movie
60625 tv
60622 tv
230131 tv
951491 movie
1399 tv
299054 movie
134095 tv
1046032 movie
603692 movie
447365 movie
502356 movie
218539 tv
95480 tv
980489 movie
385687 movie
565770 movie
76600 movie
204999 tv
120089 tv
945937 movie
121 tv
85937 tv
597 movie
671 movie
95557 tv
870358 movie
771 movie
807172 movie
1071215 movie
667538 movie
823464 movie
87917 tv
893723 movie
1047041 movie
8871 movie
968051 movie
615656 movie
1668 tv
157336 movie
1396 tv
976573 movie
120 movie
634649 movie
792307 movie
438631 movie
94997 tv
1008042 movie
11 movie
157732 tv
76479 tv
216985 tv
65494 tv
614930 movie
209867 tv
1

In [39]:
len(providers)

77

In [42]:
providers[0].keys()

dict_keys(['link', 'rent', 'flatrate', 'buy', 'id'])

In [34]:
for _, row in trending_df.iterrows():
    title = row["common_name"]

    if title not in np.unique(revenue_df["title"]):
        print(title)

Wonka
Attack on Titan
Monarch: Legacy of Monsters
Sweet Home
One Piece
Loki
Leo
Jujutsu Kaisen
Leave the World Behind
Squid Game: The Challenge
Aquaman and the Lost Kingdom
Obliterated
Family Switch
Rick and Morty
Fargo
South Sea Tomb
Game of Thrones
A Murder at the End of the World
The Bad Guys: A Very Bad Holiday
My Demon
Slow Horses
Faraway Downs
SPY x FAMILY
Fast Charlie
Demon Slayer: Kimetsu no Yaiba
Titanic
Harry Potter and the Philosopher's Stone
Merry Little Batman
Home Alone
Godzilla x Kong: The New Empire
For All Mankind
Detective Conan: Black Iron Submarine
Friends
Breaking Bad
House of the Dragon
Star Wars
Lawmen: Bass Reeves
Blood Coast
The Crown
Frieren: Beyond Journey's End
Scott Pilgrim Takes Off
Gen V
BLUE EYE SAMURAI
Bleach
The Last of Us
The Santa Clauses
